In [20]:
from __future__ import division
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [6]:
ATL = pd.read_csv('ATL.txt')
BKN = pd.read_csv('BKN.txt')
BOS = pd.read_csv('BOS.txt')
CHI = pd.read_csv('CHI.txt')
CHO = pd.read_csv('CHO.txt')
CLE = pd.read_csv('CLE.txt')
DAL = pd.read_csv('DAL.txt')
DEN = pd.read_csv('DEN.txt')
DET = pd.read_csv('DET.txt')
GSW = pd.read_csv('GSW.txt')
HOU = pd.read_csv('HOU.txt')
IND = pd.read_csv('IND.txt')
LAC = pd.read_csv('LAC.txt')
LAL = pd.read_csv('LAL.txt')
MEM = pd.read_csv('MEM.txt')
MIA = pd.read_csv('MIA.txt')
MIL = pd.read_csv('MIL.txt')
MIN = pd.read_csv('MIN.txt')
NOP = pd.read_csv('NOP.txt')
NYK = pd.read_csv('NYK.txt')
OKC = pd.read_csv('OKC.txt')
ORL = pd.read_csv('ORL.txt')
PHI = pd.read_csv('PHI.txt')
PHO = pd.read_csv('PHO.txt')
POR = pd.read_csv('POR.txt')
SAC = pd.read_csv('SAC.txt')
SAS = pd.read_csv('SAS.txt')
TOR = pd.read_csv('TOR.txt')
UTA = pd.read_csv('UTA.txt')
WAS = pd.read_csv('WAS.txt')

In [7]:
t_dict = {'Atlanta Hawks' : ATL,'Brooklyn Nets' : BKN,'Boston Celtics' : BOS,'Chicago Bulls' : CHI, 'Charlotte Hornets' : CHO,
        'Cleveland Cavaliers' : CLE,'Dallas Mavericks' : DAL, 'Denver Nuggets' : DEN,'Detroit Pistons' : DET,'Golden State Warriors' : GSW, 
        'Houston Rockets' : HOU,'Indiana Pacers' : IND,'Los Angeles Clippers' : LAC, 'Los Angeles Lakers' : LAL,'Memphis Grizzlies' : MEM,
        'Miami Heat' : MIA, 'Milwaukee Bucks' : MIL,'Minnesota Timberwolves' : MIN,'New Orleans Pelicans' : NOP, 'New York Knicks' : NYK,
        'Oklahoma City Thunder' : OKC,'Orlando Magic' : ORL, 'Philadelphia 76ers' : PHI,'Phoenix Suns' : PHO,'Portland Trail Blazers' : POR, 
        'Sacramento Kings' : SAC,'San Antonio Spurs' : SAS,'Toronto Raptors' : TOR, 'Utah Jazz' : UTA,'Washington Wizards' : WAS}

In [8]:
for team_name, team in t_dict.iteritems():
        team['Away'] = np.where(team['Unnamed: 5']=='@',1,0)
        team['Roadtrip'] = team['Away'] * (team['Away'].groupby((team['Away'] != team['Away'].shift()).cumsum()).cumcount() + 1)
        team.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4','Unnamed: 5','Unnamed: 8','Notes'], axis=1, inplace = True)
        team['diff'] = (team['Tm']-team['Opp'])
        team['+/-'] = team['diff'].cumsum()
        team['pct'] = np.round(team['W']/(team['W']+team['L']),3)
        team['win'] = np.where(team['Unnamed: 7']=='W', 1, 0)
        team['Res'] = team['Streak'].str[:1]
        team['WS'] = np.where(team['Res']=='W', 1, 0)*team['Streak'].str[2:]
        team['LS'] = np.where(team['Res']=='L', 1, 0)*team['Streak'].str[2:]
        team['WStreak'] = np.where(team['WS']=='', 0,team['WS'])
        team['LStreak'] = np.where(team['LS']=='', 0,team['LS'])
        team['WStreak'] = pd.to_numeric(team['WStreak'])
        team['LStreak'] = pd.to_numeric(team['LStreak'])
        team.drop(['Res', 'WS', 'LS','diff', 'Unnamed: 7','Streak','Tm','Opp','W','L'], axis=1, inplace = True)
        team['Prevdate'] = (pd.to_datetime(team['Date'])-pd.to_datetime(team['Date']).shift(+1)) #prev game in int
        team.drop(team.index[:1], inplace=True)
        team['Rest'] = (team['Prevdate']/ np.timedelta64(1, 'D')).astype(int)
        team.drop(['Prevdate'], axis=1, inplace = True)

In [9]:
for team_name, team in t_dict.iteritems():
    team[['Roadtrip','+/-','pct','win','WStreak','LStreak']] = \
    team[['Roadtrip','+/-','pct','win','WStreak','LStreak']].shift(+1)
    team.drop(team.index[team['Date'].str.contains('Oct')], inplace=True)
    team.drop(team.index[team['Date'].str.contains('Feb')], inplace=True)
    team.reset_index(drop=True, inplace=True)

In [10]:
for team_name, team in t_dict.iteritems():
    team['OppRoadtrip'] = np.nan
    team['Opp+/-'] = np.nan
    team['Opppct'] = np.nan
    team['Oppwin'] = np.nan
    team['OppWStreak'] = np.nan
    team['OppLStreak'] = np.nan
    team['OppRest'] = np.nan
    for index, row in team.iterrows():
        opp = row['Opponent']
        opp_team = t_dict.get(opp)
        date_comp = team.loc[index,'Date']
        opp_index_list = opp_team.index[opp_team['Date'] == date_comp].tolist()
        opp_index = 0
        opp_index = opp_index_list.pop()
        team.loc[index,'OppRoadtrip'] = opp_team.loc[opp_index,'Roadtrip']
        team.loc[index,'Opp+/-'] = opp_team.loc[opp_index,'+/-']
        team.loc[index,'Opppct'] = opp_team.loc[opp_index,'pct']
        team.loc[index,'Oppwin'] = opp_team.loc[opp_index,'win']
        team.loc[index,'OppWStreak'] = opp_team.loc[opp_index,'WStreak']
        team.loc[index,'OppLStreak'] = opp_team.loc[opp_index,'LStreak']
        team.loc[index,'OppRest'] = opp_team.loc[opp_index,'Rest']

In [11]:
data = pd.DataFrame(columns = ['Away','Roadtrip','+/-','pct','win','WStreak','LStreak','Rest','OppRoadtrip',\
                              'Opp+/-','Opppct','Oppwin','OppWStreak','OppLStreak','OppRest'])
for team_name, team in t_dict.iteritems():
    team.drop(['G','Date','Opponent'], axis=1, inplace = True)
    data = pd.concat([data, team], ignore_index=True)

In [12]:
###RUN TO OBTAIN LABELS###

In [13]:
ATL = pd.read_csv('ATL.txt')
BKN = pd.read_csv('BKN.txt')
BOS = pd.read_csv('BOS.txt')
CHI = pd.read_csv('CHI.txt')
CHO = pd.read_csv('CHO.txt')
CLE = pd.read_csv('CLE.txt')
DAL = pd.read_csv('DAL.txt')
DEN = pd.read_csv('DEN.txt')
DET = pd.read_csv('DET.txt')
GSW = pd.read_csv('GSW.txt')
HOU = pd.read_csv('HOU.txt')
IND = pd.read_csv('IND.txt')
LAC = pd.read_csv('LAC.txt')
LAL = pd.read_csv('LAL.txt')
MEM = pd.read_csv('MEM.txt')
MIA = pd.read_csv('MIA.txt')
MIL = pd.read_csv('MIL.txt')
MIN = pd.read_csv('MIN.txt')
NOP = pd.read_csv('NOP.txt')
NYK = pd.read_csv('NYK.txt')
OKC = pd.read_csv('OKC.txt')
ORL = pd.read_csv('ORL.txt')
PHI = pd.read_csv('PHI.txt')
PHO = pd.read_csv('PHO.txt')
POR = pd.read_csv('POR.txt')
SAC = pd.read_csv('SAC.txt')
SAS = pd.read_csv('SAS.txt')
TOR = pd.read_csv('TOR.txt')
UTA = pd.read_csv('UTA.txt')
WAS = pd.read_csv('WAS.txt')

In [14]:
#READ DATA AGAIN
label_dict = {'Atlanta Hawks' : ATL,'Brooklyn Nets' : BKN,'Boston Celtics' : BOS,'Chicago Bulls' : CHI, 'Charlotte Hornets' : CHO,
        'Cleveland Cavaliers' : CLE,'Dallas Mavericks' : DAL, 'Denver Nuggets' : DEN,'Detroit Pistons' : DET,'Golden State Warriors' : GSW, 
        'Houston Rockets' : HOU,'Indiana Pacers' : IND,'Los Angeles Clippers' : LAC, 'Los Angeles Lakers' : LAL,'Memphis Grizzlies' : MEM,
        'Miami Heat' : MIA, 'Milwaukee Bucks' : MIL,'Minnesota Timberwolves' : MIN,'New Orleans Pelicans' : NOP, 'New York Knicks' : NYK,
        'Oklahoma City Thunder' : OKC,'Orlando Magic' : ORL, 'Philadelphia 76ers' : PHI,'Phoenix Suns' : PHO,'Portland Trail Blazers' : POR, 
        'Sacramento Kings' : SAC,'San Antonio Spurs' : SAS,'Toronto Raptors' : TOR, 'Utah Jazz' : UTA,'Washington Wizards' : WAS}

In [15]:
for team_name, team in label_dict.iteritems():
        team['Away'] = np.where(team['Unnamed: 5']=='@',1,0)
        team['Roadtrip'] = team['Away'] * (team['Away'].groupby((team['Away'] != team['Away'].shift()).cumsum()).cumcount() + 1)
        team.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4','Unnamed: 5','Unnamed: 8','Notes'], axis=1, inplace = True)
        team['diff'] = (team['Tm']-team['Opp'])
        team['+/-'] = team['diff'].cumsum()
        team['pct'] = np.round(team['W']/(team['W']+team['L']),3)
        team['win'] = np.where(team['Unnamed: 7']=='W', 1, 0)
        team['Res'] = team['Streak'].str[:1]
        team['WS'] = np.where(team['Res']=='W', 1, 0)*team['Streak'].str[2:]
        team['LS'] = np.where(team['Res']=='L', 1, 0)*team['Streak'].str[2:]
        team['WStreak'] = np.where(team['WS']=='', 0,team['WS'])
        team['LStreak'] = np.where(team['LS']=='', 0,team['LS'])
        team['WStreak'] = pd.to_numeric(team['WStreak'])
        team['LStreak'] = pd.to_numeric(team['LStreak'])
        team.drop(['Res', 'WS', 'LS','diff', 'Unnamed: 7','Streak','Tm','Opp','W','L'], axis=1, inplace = True)
        team['Prevdate'] = (pd.to_datetime(team['Date'])-pd.to_datetime(team['Date']).shift(+1)) #prev game in int
        team.drop(team.index[:1], inplace=True)
        team['Rest'] = (team['Prevdate']/ np.timedelta64(1, 'D')).astype(int)
        team.drop(['Prevdate'], axis=1, inplace = True)

In [16]:
for team_name, team in label_dict.iteritems():
    team.drop(team.index[team['Date'].str.contains('Oct')], inplace=True)
    team.drop(team.index[team['Date'].str.contains('Feb')], inplace=True)
    team.drop(['G','Date', 'Opponent','Away', 'Roadtrip','+/-', 'pct','WStreak','LStreak','Rest'], axis=1, inplace = True)
    team.reset_index(drop=True, inplace=True)

In [17]:
labels = pd.DataFrame(columns = ['win'])
for team_name, team in label_dict.iteritems():
    labels = pd.concat([labels, team], ignore_index=True)

In [ ]:
#####GETTING DATA READY FOR MODELLING

In [105]:
X = data.values
y = labels.values
y=y.astype('int')
X_scaled = preprocessing.scale(X)

In [19]:
X_train = np.asarray(X_scaled[:1000],dtype=np.float64)
X_test = np.asarray(X_scaled[1000:],dtype=np.float64)
y_train = np.asarray(y[:1000],dtype=np.float64)
y_test = np.asarray(y[1000:],dtype=np.float64)

In [236]:
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [208]:
nn = MLPClassifier(activation='identity',alpha=0.001, #identiy seems to be working, large regularisation(gradient on full set)
                    batch_size=X.shape[0], #small dataset, optimising on full
                    hidden_layer_sizes=(30,50,30), #heuristically chosen
                    learning_rate_init=0.001, max_iter=100, #small learning rate and many iterations(small dataset)
                    solver='sgd') #stochastic grdient descent chosen(simplest method - no need for complexity)
scores = cross_val_score(nn, X_scaled, y.ravel(), cv=10, scoring='accuracy') #10 fold - split ~ 1180/130
print scores.mean()

0.618251748252


In [235]:
rf = RandomForestClassifier(max_depth=5, random_state=0) #don't allow trees to go deep, prevent overfitting
scores = cross_val_score(rf, X_scaled, y.ravel(), cv=10, scoring='accuracy') #10 fold - split ~ 1180/130
print scores.mean()

0.620454545455


In [254]:
clf = linear_model.SGDClassifier(loss = 'squared_loss')
scores = cross_val_score(clf, X_scaled, y.ravel(), cv=10, scoring='accuracy') #10 fold - split ~ 1180/130
print scores.mean()

0.522179487179
